In [1]:
from tensorflow import keras
from keras import layers
import matplotlib.pyplot as plt
import numpy as np

In [2]:
data = np.load('../../data/othello_train_data.npz' ,allow_pickle=True)
X, y = data['X'], data['y']
X = np.where(X == 3, 0, X)



In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = scaler.fit_transform(X_train.reshape(-1, 64)).reshape(-1, 8, 8)
X_val = scaler.transform(X_val.reshape(-1, 64)).reshape(-1, 8, 8)

In [4]:
print("Training data shape:", X_train.shape, y_train.shape)
print("-" *10)

print(X_train[50])
print(f'This game winner was {y_train[50]}')

Training data shape: (47104, 8, 8) (47104,)
----------
[[ 1.  1. -1.  0.  1.  0. -1.  1.]
 [-1.  1.  0.  1.  1.  1.  0.  1.]
 [-1. -1.  1.  0.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  0.  0.  1.]
 [-1. -1.  1.  1.  1.  1.  0.  1.]
 [-1. -1.  1.  1.  1.  0.  0.  0.]
 [ 0.  0.  0.  1.  1.  1. -1.  1.]
 [-1.  0. -1.  0. -1.  1. -1. -1.]]
This game winner was -1


In [16]:
network = keras.Sequential([
    layers.Input(shape=(8,8,1)),

    layers.Conv2D(32, (3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),
    
    layers.Conv2D(32, (3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),

    layers.Conv2D(32, (3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),

    layers.Flatten(),
    layers.Dense(64, kernel_regularizer=keras.regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.ReLU(),


    layers.Dense(1, activation='tanh')
    ])
network.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),loss='mse',metrics=['r2_score'])

In [17]:
early_stop = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)


lr_scheduler = keras.callbacks.ReduceLROnPlateau(
    monitor='val_r2_score',     
    factor=0.5,              
    patience=5,              
    min_lr=1e-5,             
    verbose=1
)


history = network.fit(X_train, y_train, epochs=200, batch_size=32,validation_split=0.2,callbacks=[early_stop,lr_scheduler])

Epoch 1/200
1178/1178 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 0.9347 - r2_score: -0.0103 - val_loss: 0.7054 - val_r2_score: 0.2261 - learning_rate: 0.0010
Epoch 2/200
1178/1178 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 0.6762 - r2_score: 0.2677 - val_loss: 0.6522 - val_r2_score: 0.2947 - learning_rate: 0.0010
Epoch 3/200
1178/1178 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 0.5598 - r2_score: 0.4100 - val_loss: 0.5387 - val_r2_score: 0.4411 - learning_rate: 0.0010
Epoch 4/200
1178/1178 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.4878 - r2_score: 0.5032 - val_loss: 0.5406 - val_r2_score: 0.4441 - learning_rate: 0.0010
Epoch 5/200
1178/1178 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 0.4461 - r2_score: 0.5615 - val_loss: 0.4629 - val_r2_score: 0.5409 - learning_rate: 0.0010
Epoch 6/200
1168/1178 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.4063 - r2_score: 0.6113
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
1178/1178 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.4064 -

In [18]:
network.evaluate(X_val, y_val)

368/368 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2340 - r2_score: 0.7428


[0.23205773532390594, 0.7460777759552002]

In [ ]:
#Uncomment below to save the  model
#network.save("../../models/model_v3.keras")